<a href="https://colab.research.google.com/github/sajacaros/optiver/blob/main/sajacaros/02_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq timm torchmetrics torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00


In [2]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings(action='ignore')
# warnings.filterwarnings(action='default')

In [3]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../../data/'

Mounted at /content/drive


In [4]:
train_df = pd.read_csv(data_path+'train.csv')
test_df = pd.read_csv(data_path+'test.csv')
submission_df = pd.read_csv(data_path+'sample_submission.csv')
target_df = pd.read_csv(data_path+'revealed_targets.csv')

# <추가적인 피쳐 정보>
* stock: 200개
* 피쳐수: 16개 + 타겟 1개
* train 기간: 0~480, 481일
* seconds_in_bucket
    * 0~540 (10씩 증가)
    * stock당 하루 55개의 data가 쌓임
    * 300부터는 far_price와 near_price가 존재함
* 일부 stock의 경우 데이터가 누락됨
    * 종목당 55*481 = 26455 개의 데이터가 있어야 함
    * 11개의 종목이 데이터가 누락됨(아래 그림 참고)
    * 총 length가 5,291,000(200*55*541)이 되어야 하지만 누락건수로 인해 5,237,980이 됨

In [5]:
# null 값 처리
train_df['far_price'] = train_df['far_price'].fillna(0)
train_df['near_price'] = train_df['near_price'].fillna(1)

In [6]:
train_df.loc[train_df.isnull().any(axis=1)] # 220

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,NaN,0,NaN,NaN,0.0,1.0,NaN,0.00,NaN,0.00,NaN,NaN,1925,35_0_131
369700,131,35,10,NaN,0,NaN,NaN,0.0,1.0,NaN,0.00,NaN,0.00,NaN,NaN,1926,35_10_131
369892,131,35,20,NaN,0,NaN,NaN,0.0,1.0,NaN,0.00,NaN,0.00,NaN,NaN,1927,35_20_131
370084,131,35,30,NaN,0,NaN,NaN,0.0,1.0,NaN,0.00,NaN,0.00,NaN,NaN,1928,35_30_131
370276,131,35,40,NaN,0,NaN,NaN,0.0,1.0,NaN,0.00,NaN,0.00,NaN,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774999,19,438,500,NaN,-1,NaN,NaN,0.0,1.0,NaN,115491.18,NaN,1990.10,NaN,3.319979,24140,438_500_19
4775199,19,438,510,NaN,-1,NaN,NaN,0.0,1.0,NaN,189040.50,NaN,26283.84,NaN,-5.049705,24141,438_510_19
4775399,19,438,520,NaN,-1,NaN,NaN,0.0,1.0,NaN,1392.93,NaN,43799.80,NaN,-0.110269,24142,438_520_19
4775599,19,438,530,NaN,-1,NaN,NaN,0.0,1.0,NaN,13531.32,NaN,26881.20,NaN,-1.689792,24143,438_530_19


In [7]:
cols_group_by = ['date_id', 'seconds_in_bucket']
cols_fill_nan = [
    'imbalance_size', 'reference_price', 'matched_size', 'wap',
    'bid_price', 'bid_size', 'ask_price', 'ask_size',
    'stock_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag']
train_grouped_median = train_df.groupby(cols_group_by)[cols_fill_nan].transform('median')
train_df[cols_fill_nan] = train_df[cols_fill_nan].fillna(train_grouped_median)
train_df.loc[train_df.isnull().any(axis=1)] # target 88

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,1381981.10,0,0.999950,9723622.86,0.0,1.0,0.999688,0.0,1.000242,0.0,1.000000,NaN,1925,35_0_131
369700,131,35,10,1371886.54,0,1.000252,9961197.49,0.0,1.0,0.999969,0.0,1.000485,0.0,1.000223,NaN,1926,35_10_131
369892,131,35,20,1331838.54,0,1.000122,9999133.11,0.0,1.0,0.999883,0.0,1.000328,0.0,1.000149,NaN,1927,35_20_131
370084,131,35,30,1350584.58,0,0.999910,10133596.07,0.0,1.0,0.999757,0.0,1.000186,0.0,0.999971,NaN,1928,35_30_131
370276,131,35,40,1327284.70,0,0.999926,10133596.07,0.0,1.0,0.999758,0.0,1.000203,0.0,0.999984,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225338,158,388,510,339056.84,0,1.001074,19640140.62,0.0,1.0,1.000867,0.0,1.001250,0.0,1.001027,NaN,21391,388_510_158
4225538,158,388,520,437746.70,0,1.001183,19767348.90,0.0,1.0,1.001007,0.0,1.001290,0.0,1.001163,NaN,21392,388_520_158
4225738,158,388,530,483535.46,0,1.000994,19800447.29,0.0,1.0,1.000905,0.0,1.001187,0.0,1.001032,NaN,21393,388_530_158
4225938,158,388,540,453894.62,0,1.000989,20020720.89,0.0,1.0,1.000824,0.0,1.001024,0.0,1.000911,NaN,21394,388_540_158


In [8]:
train_df = train_df.dropna()

In [9]:
selected_features = [
    'date_id', 'seconds_in_bucket', 'imbalance_size', 'reference_price', 'matched_size', 'wap',
    'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size',
    'stock_id', 'imbalance_buy_sell_flag']

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
X_all = scaler.fit_transform(train_df[selected_features].values)
y_all = train_df[['target']].values

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision.transforms import transforms

In [13]:
# 장비 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
train_dataset = TensorDataset(torch.HalfTensor(X_train), torch.HalfTensor(y_train))
valid_dataset = TensorDataset(torch.HalfTensor(X_valid), torch.HalfTensor(y_valid))

In [15]:
batch_size=4096
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [16]:
input_size = len(selected_features)
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=input_size*2) # input => hidden1
        self.fc2 = nn.Linear(in_features=input_size*2, out_features=input_size) # hidden1 => hidden2
        self.fc3 = nn.Linear(in_features=input_size, out_features=1) # hidden3 => output
        self.relu = nn.ReLU() # activation layer

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model().to(device)
model

Model(
  (fc1): Linear(in_features=14, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=14, bias=True)
  (fc3): Linear(in_features=14, out_features=1, bias=True)
  (relu): ReLU()
)

In [17]:
# 손실 함수
criterion = nn.L1Loss()

In [18]:
# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [19]:
from tqdm.auto import tqdm
epochs = 30

for epoch in range(epochs):

    train_epoch_loss = []
    bar = tqdm(enumerate(train_loader), total = len(train_loader), desc='Train Loop')
    model.train()
    for idx, (stocks, movements) in bar:
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            stocks = stocks.to(device)
            movements = movements.to(device)
            outputs = model(stocks)

            loss = criterion(outputs.view(-1), movements)
            train_epoch_loss.append(loss.item())
            optimizer.zero_grad() # 기울기 초기화
            loss.backward()
            optimizer.step()
            bar.set_postfix(
                Epoch = epoch,
                Train_loss = loss.item(),#sum(train_epoch_loss)/(idx+1),
                LR = optimizer.param_groups[0]['lr'],
            )
    print(f'Epoch [{epoch+1}/{epochs}] - Train loss : {sum(train_epoch_loss)/len(train_loader):.4f}')

    model.eval()
    valid_epoch_loss = []
    bar = tqdm(enumerate(valid_loader), total = len(valid_loader), desc='Valid Loop')
    for idx, (stocks, movements) in bar:
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            with torch.no_grad():
                stocks = stocks.to(device)
                movements = movements.to(device)
                outputs = model(stocks)
                loss = criterion(outputs.view(-1), movements)
                valid_epoch_loss.append(loss.item())
    print(f'Epoch [{epoch+1}/{epochs}] - Valid loss : {sum(valid_epoch_loss)/len(valid_loader):.4f}')

Train Loop:   0%|          | 0/1023 [00:00<?, ?it/s]

Epoch [1/30] - Train loss : 6.4083


Valid Loop:   0%|          | 0/255 [00:00<?, ?it/s]

Epoch [1/30] - Valid loss : 6.4044


Train Loop:   0%|          | 0/1023 [00:00<?, ?it/s]

...

Epoch [29/30] - Valid loss : 6.4044


Train Loop:   0%|          | 0/1023 [00:00<?, ?it/s]

Epoch [30/30] - Train loss : 6.4083


Valid Loop:   0%|          | 0/255 [00:00<?, ?it/s]

Epoch [30/30] - Valid loss : 6.4044
